In [2]:
cd ..

/Users/nicolevandeweijer/PycharmProjects/ma-communicative-robots-fork/projects/memory


In [3]:
import torch
import warnings

warnings.filterwarnings("ignore")
import nltk

nltk.download("all")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import gender_guesser.detector as gender

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/nicolevandeweijer/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/nicolevandeweijer/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /Users/nicolevandeweijer/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /Users/nicolevandeweijer/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /Users/nicolevandeweijer/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /Users/nicolevandeweijer/nltk_data...
[n

[nltk_data]    |   Package toolbox is already up-to-date!
[nltk_data]    | Downloading package treebank to
[nltk_data]    |     /Users/nicolevandeweijer/nltk_data...
[nltk_data]    |   Package treebank is already up-to-date!
[nltk_data]    | Downloading package twitter_samples to
[nltk_data]    |     /Users/nicolevandeweijer/nltk_data...
[nltk_data]    |   Package twitter_samples is already up-to-date!
[nltk_data]    | Downloading package udhr to
[nltk_data]    |     /Users/nicolevandeweijer/nltk_data...
[nltk_data]    |   Package udhr is already up-to-date!
[nltk_data]    | Downloading package udhr2 to
[nltk_data]    |     /Users/nicolevandeweijer/nltk_data...
[nltk_data]    |   Package udhr2 is already up-to-date!
[nltk_data]    | Downloading package unicode_samples to
[nltk_data]    |     /Users/nicolevandeweijer/nltk_data...
[nltk_data]    |   Package unicode_samples is already up-to-date!
[nltk_data]    | Downloading package universal_treebanks_v20 to
[nltk_data]    |     /Users/n

In [4]:
import json
from glob import glob
from tqdm.notebook import tqdm
import os
import random
import re
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

tokenizer = T5Tokenizer.from_pretrained("tscholak/t5.1.1.lm100k.base")
model = T5ForConditionalGeneration.from_pretrained("tscholak/t5.1.1.lm100k.base")


def read_json(path):
    """Read json file.

    Args
    ----
    path: path to the json

    Returns
    -------
    loaded: loaded dict

    """
    with open(path, "r") as stream:
        loaded = json.load(stream)

    return loaded


def atoi(text):
    return int(text) if text.isdigit() else text


def natural_keys(text):
    """
    copied from https://stackoverflow.com/questions/5967500/how-to-correctly-sort-a-string-with-a-number-inside
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)

    """
    return [atoi(c) for c in re.split(r"(\d+)", text)]

In [5]:
def random_state(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


random_state(1234)

In [6]:
def convert_to_days(entry):
    entry = sorted(entry, key=lambda x: x[-1])
    for idx, mem in enumerate(entry):
        max_len = len(entry)
        days = len(entry) - idx - 1
        if days == 0:
            timestamp = "today"
        else:
            timestamp = f"{days} days ago"
        entry[idx][-1] = timestamp

In [8]:
def calculate_accuracy(answers, predictions):
    correct = 0
    wrong = 0
    for answer, pred in zip(answers, predictions):
        if answer in pred:
            correct += 1
        else:
            wrong += 1

    print(f"correct: {correct}\nwrong: {wrong}")
    print(f"accuracy is {correct / (correct+ wrong)*100}")

    accuracy = correct / (correct + wrong) * 100
    return correct, wrong, accuracy

## Change model here:

In [7]:
def predict(prompts):
    predictions = []
    for prompt in prompts:
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids
        outputs = model.generate(input_ids)
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
        #         print(prediction)
        predictions.append(prediction)
    return predictions

# Prompts

### Prompt 1: Baseline

Example: Mary's bench was at Mary's city, today. Where is Mary's bench?

In [ ]:
corrects = []
wrongs = []
accuracies = []

paths = glob("data/*.json")
paths.sort(key=natural_keys)
for path in tqdm(paths):
    data = read_json(path)
    print(path)
    prompts = []
    answers = []
    for entry in data["val"]:
        entry["episodic_memory_system"] = sorted(
            entry["episodic_memory_system"], key=lambda x: x[-1]
        )
        convert_to_days(entry["episodic_memory_system"])

        prompt = []
        for mem in entry["episodic_memory_system"]:
            prompt.append(f"{mem[0]} was at {mem[2]}, {mem[3]}.")
        for mem in entry["semantic_memory_system"]:
            prompt.append(f"{mem[-1]} {mem[0]} were found at {mem[2]}.")
        prompt.append(f"Where is {entry['question'][0]}?")
        prompt = " ".join(prompt)
        prompts.append(prompt)
        #         print(prompt)
        answers.append(entry["correct_answer"])
    predictions = predict(prompts)

    correct, wrong, accuracy = calculate_accuracy(answers, predictions)
    corrects.append(correct)
    wrongs.append(wrong)
    accuracies.append(accuracy)

    print("\n\n")

prompt1 = [
    {"Correct": correct, "Wrong": wrong, "Accuracy": accuracy}
    for correct, wrong, accuracy in zip(corrects, wrongs, accuracies)
]
print(prompt1)

### Prompt 2: No question, only subject

Example: Mary's bench was at Mary's city, today. Mary's bench

In [ ]:
corrects = []
wrongs = []
accuracies = []

paths = glob("data/*.json")
paths.sort(key=natural_keys)
for path in tqdm(paths):
    data = read_json(path)
    print(path)
    prompts = []
    answers = []
    for entry in data["val"]:
        entry["episodic_memory_system"] = sorted(
            entry["episodic_memory_system"], key=lambda x: x[-1]
        )
        convert_to_days(entry["episodic_memory_system"])

        prompt = []
        for mem in entry["episodic_memory_system"]:
            prompt.append(f"{mem[0]} was at {mem[2]}, {mem[3]}.")
        for mem in entry["semantic_memory_system"]:
            prompt.append(f"{mem[-1]} {mem[0]} were found at {mem[2]}.")
        prompt.append(entry["question"][0])
        prompt = " ".join(prompt)
        prompts.append(prompt)
        #         print(prompt)
        answers.append(entry["correct_answer"])
    predictions = predict(prompts)

    correct, wrong, accuracy = calculate_accuracy(answers, predictions)
    corrects.append(correct)
    wrongs.append(wrong)
    accuracies.append(accuracy)

    print("\n\n")

prompt2 = [
    {"Correct": correct, "Wrong": wrong, "Accuracy": accuracy}
    for correct, wrong, accuracy in zip(corrects, wrongs, accuracies)
]
print(prompt2)

### Prompt 3: prompt with more guidance

Memories: Mary's bench was at Mary's city, today. 

Question: Where is Mary's bench? 

Answer:

In [ ]:
corrects = []
wrongs = []
accuracies = []

paths = glob("data/*.json")
paths.sort(key=natural_keys)
for path in tqdm(paths):
    data = read_json(path)
    print(path)
    prompts = []
    answers = []
    for entry in data["val"]:
        entry["episodic_memory_system"] = sorted(
            entry["episodic_memory_system"], key=lambda x: x[-1]
        )
        convert_to_days(entry["episodic_memory_system"])

        prompt = []
        prompt.append(f"Memories:")
        for mem in entry["episodic_memory_system"]:
            prompt.append(f"{mem[0]} was at {mem[2]}, {mem[3]}.")
        for mem in entry["semantic_memory_system"]:
            prompt.append(f"{mem[-1]} {mem[0]} were found at {mem[2]}.")
        prompt.append(f"\nQuestion: Where is {entry['question'][0]}? \nAnswer:")
        prompt = " ".join(prompt)
        prompts.append(prompt)
        #         print(prompt)
        answers.append(entry["correct_answer"])
    predictions = predict(prompts)
    #     print("\n")

    correct, wrong, accuracy = calculate_accuracy(answers, predictions)
    corrects.append(correct)
    wrongs.append(wrong)
    accuracies.append(accuracy)

    print("\n\n")

prompt3 = [
    {"Correct": correct, "Wrong": wrong, "Accuracy": accuracy}
    for correct, wrong, accuracy in zip(corrects, wrongs, accuracies)
]
print(prompt3)

### Prompt 4: Generalized episodic memories

Example: The bench was at the city, today. Where is Mary's bench?

In [ ]:
corrects = []
wrongs = []
accuracies = []

paths = glob("data/*.json")
paths.sort(key=natural_keys)
for path in tqdm(paths):
    data = read_json(path)
    print(path)
    prompts = []
    answers = []
    for entry in data["val"]:
        entry["episodic_memory_system"] = sorted(
            entry["episodic_memory_system"], key=lambda x: x[-1]
        )
        convert_to_days(entry["episodic_memory_system"])

        prompt = []
        for mem in entry["episodic_memory_system"]:
            _subject = mem[0].split(" ")[1]
            _object = mem[2].split(" ")[1]
            prompt.append(f"The {_subject} was at the {_object}, {mem[3]}.")
        for mem in entry["semantic_memory_system"]:
            prompt.append(f"{mem[-1]} {mem[0]} were found at {mem[2]}.")
        prompt.append(f"Where is {entry['question'][0]}?")

        prompt = " ".join(prompt)
        prompts.append(prompt)
        #         print(prompt)
        answers.append(entry["correct_answer"])
    predictions = predict(prompts)
    #     print("\n")

    correct, wrong, accuracy = calculate_accuracy(answers, predictions)
    corrects.append(correct)
    wrongs.append(wrong)
    accuracies.append(accuracy)

    print("\n\n")

prompt4 = [
    {"Correct": correct, "Wrong": wrong, "Accuracy": accuracy}
    for correct, wrong, accuracy in zip(corrects, wrongs, accuracies)
]
print(prompt4)

# Results

In [ ]:
accuracy1 = []
for i in prompt1:
    accuracy1.append(i.get("Accuracy"))

accuracy2 = []
for i in prompt2:
    accuracy2.append(i.get("Accuracy"))

accuracy3 = []
for i in prompt3:
    accuracy3.append(i.get("Accuracy"))

accuracy4 = []
for i in prompt4:
    accuracy4.append(i.get("Accuracy"))

Which prompt results in best accuracy?

In [ ]:
df = pd.DataFrame(
    {
        "file": paths,
        "prompt 1": accuracy1,
        "prompt 2": accuracy2,
        "prompt 3": accuracy3,
        "prompt 4": accuracy4,
    }
).T
df = df.rename(columns=df.iloc[0]).drop(df.index[0])
df = df.style.highlight_max(color="lightblue")
df

Which file results in best accuracy?

In [ ]:
df2 = pd.DataFrame(
    {
        "file": paths,
        "prompt 1": accuracy1,
        "prompt 2": accuracy2,
        "prompt 3": accuracy3,
        "prompt 4": accuracy4,
    }
)
df2 = df2.set_index("file")
df2 = df2.style.highlight_max(color="lightblue")
df2